# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [13]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [14]:
total_schools = school_data_complete['school_name'].value_counts().count()
total_schools

15

In [15]:
total_students = school_data_complete['Student ID'].value_counts().count()
total_students

39170

In [16]:
total_budget = school_data['budget'].sum()
avg_reading_score = round(school_data_complete['reading_score'].mean(), 2)
avg_math_score = round(school_data_complete['math_score'].mean(), 2)

print(total_budget," ", avg_reading_score," ", avg_math_score )

24649428   81.88   78.99


In [17]:
pass_read = school_data_complete[school_data_complete['reading_score']>= 70]['Student ID'].count()
pass_math = school_data_complete[school_data_complete['math_score']>=70]['Student ID'].count()
#pass_both = school_data_complete[(school_data_complete['reading_score']>=70) & (school_data_complete['math_score']>=70)]['Student ID'].count()

percent_pass_read = 100*(pass_read / total_students)
percent_pass_math = 100*(pass_math / total_students)

percent_pass_overall = (percent_pass_read + percent_pass_math)/2

print(percent_pass_read, " " ,percent_pass_math, " ", percent_pass_overall)

85.80546336482001   74.9808526933878   80.39315802910392


In [18]:
district_summary =pd.DataFrame({'Total Schools': total_schools,
              'Total Students': total_students,
              'Total Budget': total_budget,
              'Average Math Score': avg_math_score,
              'Average Reading Score': avg_reading_score,
              '% Passing Math': percent_pass_math,
              '% Passing Reading': percent_pass_read,
              '% Overall Passing Rate': percent_pass_overall 
             }, index = [0])

district_summary['Total Budget'] = district_summary['Total Budget'].map('${:,.2f}'.format)
district_summary['% Passing Math'] = district_summary['% Passing Math'].map('{:.2f}'.format)
district_summary['% Passing Reading'] = district_summary['% Passing Reading'].map('{:.2f}'.format)
district_summary['% Overall Passing Rate'] = district_summary['% Overall Passing Rate'].map('{:.2f}'.format)


In [19]:
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.99,81.88,74.98,85.81,80.39


* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [20]:
#school_data_complete.head()

In [21]:
# total count of students by school
df0 = school_data_complete.groupby('school_name').count()[['Student ID']].rename(columns={'Student ID': 'Total Students'})

In [22]:
# total budget of students by school 
#df1 = school_data_complete.groupby('school_name').sum()[['budget']].rename(columns={'budget': 'Total Budget'})

In [23]:
# average of reading and math score by school
df2 = school_data_complete.groupby('school_name').mean()[['reading_score', 'math_score']].rename(columns={'reading_score': 'Average Reading Score',
                                                                                                    'math_score': 'Average Math Score'})

In [24]:
# get list of school names, type and budget
df3 = school_data_complete[['school_name', 'type','budget']].drop_duplicates().rename(columns={'type': 'School Type', 'budget': 'Total School Budget'}).set_index('school_name')

In [25]:
# number of students per school that passed reading
df4_read = pd.DataFrame(school_data_complete[school_data_complete['reading_score']>=70].groupby('school_name').count()['reading_score'])

In [26]:
# number of students per school that passed math
df4_math = pd.DataFrame(school_data_complete[school_data_complete['math_score']>=70].groupby('school_name').count()['math_score'])

In [27]:
school_summary = pd.merge(df0, df2, on='school_name')
school_summary = pd.merge(school_summary, df3, on='school_name')
school_summary = pd.merge(school_summary, df4_read, on='school_name')
school_summary = pd.merge(school_summary, df4_math, on='school_name')

In [28]:
#school_summary

In [29]:
school_summary['% Passing Reading']= 100*(school_summary['reading_score']/school_summary['Total Students'])
school_summary['% Passing Math']= 100*(school_summary['math_score']/school_summary['Total Students'])

school_summary['% Overall Passing Rate'] = (school_summary['% Passing Reading'] + school_summary['% Passing Math'])/2
school_summary['Per Student Budget'] = school_summary['Total School Budget']/school_summary['Total Students']

del school_summary['reading_score']
del school_summary['math_score']

In [30]:
#school_summary

In [31]:
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget','Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading','% Overall Passing Rate']]

In [32]:
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992


In [33]:
top_schools = school_summary.sort_values('% Overall Passing Rate', ascending = False)
bottom_schools = school_summary.sort_values('% Overall Passing Rate', ascending = True)



In [34]:
top_schools['Total School Budget']=top_schools['Total School Budget'].map('${:,.2f}'.format)
top_schools['Average Reading Score']=top_schools['Average Reading Score'].map('{:.2f}'.format)
top_schools['Average Math Score']=top_schools['Average Math Score'].map('{:.2f}'.format)
top_schools['% Passing Reading']=top_schools['% Passing Reading'].map('{:.2f}'.format)
top_schools['% Passing Math']=top_schools['% Passing Math'].map('{:.2f}'.format)
top_schools['% Overall Passing Rate']=top_schools['% Overall Passing Rate'].map('{:.2f}'.format)


In [35]:
bottom_schools['Total School Budget']=bottom_schools['Total School Budget'].map('${:,.2f}'.format)
bottom_schools['Average Reading Score']=bottom_schools['Average Reading Score'].map('{:.2f}'.format)
bottom_schools['Average Math Score']=bottom_schools['Average Math Score'].map('{:.2f}'.format)
bottom_schools['% Passing Reading']=bottom_schools['% Passing Reading'].map('{:.2f}'.format)
bottom_schools['% Passing Math']=bottom_schools['% Passing Math'].map('{:.2f}'.format)
bottom_schools['% Overall Passing Rate']=bottom_schools['% Overall Passing Rate'].map('{:.2f}'.format)

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [36]:
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.06,83.98,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,043,130.00",638.0,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,962,"$585,858.00",609.0,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,1468,"$917,500.00",625.0,83.35,83.82,93.39,97.14,95.27
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.27,83.99,93.87,96.54,95.20
Wright High School,Charter,1800,"$1,049,400.00",583.0,83.68,83.95,93.33,96.61,94.97
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.36,83.73,93.87,95.85,94.86
Holden High School,Charter,427,"$248,087.00",581.0,83.80,83.81,92.51,96.25,94.38
Bailey High School,District,4976,"$3,124,928.00",628.0,77.05,81.03,66.68,81.93,74.31


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [37]:
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",637.0,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.71,81.16,65.99,80.74,73.36
Huang High School,District,2917,"$1,910,635.00",655.0,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",650.0,77.07,80.97,66.06,81.22,73.64
Ford High School,District,2739,"$1,763,916.00",644.0,77.10,80.75,68.31,79.30,73.80
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.29,80.93,66.75,80.86,73.81
Bailey High School,District,4976,"$3,124,928.00",628.0,77.05,81.03,66.68,81.93,74.31
Holden High School,Charter,427,"$248,087.00",581.0,83.80,83.81,92.51,96.25,94.38
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.36,83.73,93.87,95.85,94.86


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [38]:
#school_data_complete.head()

In [39]:
freshmen_math = school_data_complete[['school_name','grade', 'math_score']][school_data_complete['grade']=='9th'].groupby('school_name').mean()
sphomore_math = school_data_complete[['school_name','grade', 'math_score']][school_data_complete['grade']=='10th'].groupby('school_name').mean()
junior_math = school_data_complete[['school_name','grade', 'math_score']][school_data_complete['grade']=='11th'].groupby('school_name').mean()
senior_math = school_data_complete[['school_name','grade', 'math_score']][school_data_complete['grade']=='12th'].groupby('school_name').mean()

In [40]:
freshmen_math = freshmen_math.rename(columns={'math_score': '9th Grade Avg Math Score'})
sphomore_math = sphomore_math.rename(columns={'math_score': '10th Grade Avg Math Score'})
junior_math = junior_math.rename(columns={'math_score': '11th Grade Avg Math Score'})
senior_math = senior_math.rename(columns={'math_score': '12th Grade Avg Math Score'})

Avg_math_by_school = pd.merge(freshmen_math, sphomore_math, on='school_name')
Avg_math_by_school = pd.merge(Avg_math_by_school, junior_math, on='school_name')
Avg_math_by_school = pd.merge(Avg_math_by_school, senior_math, on='school_name')

Avg_math_by_school['9th Grade Avg Math Score'] = Avg_math_by_school['9th Grade Avg Math Score'].map('{:.2f}'.format)
Avg_math_by_school['10th Grade Avg Math Score'] = Avg_math_by_school['10th Grade Avg Math Score'].map('{:.2f}'.format)
Avg_math_by_school['11th Grade Avg Math Score'] = Avg_math_by_school['11th Grade Avg Math Score'].map('{:.2f}'.format)
Avg_math_by_school['12th Grade Avg Math Score'] = Avg_math_by_school['12th Grade Avg Math Score'].map('{:.2f}'.format)

In [41]:
Avg_math_by_school

,9th Grade Avg Math Score,10th Grade Avg Math Score,11th Grade Avg Math Score,12th Grade Avg Math Score
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [42]:
freshmen_rdn = school_data_complete[['school_name','grade', 'reading_score']][school_data_complete['grade']=='9th'].groupby('school_name').mean()
sphomore_rdn = school_data_complete[['school_name','grade', 'reading_score']][school_data_complete['grade']=='10th'].groupby('school_name').mean()
junior_rdn = school_data_complete[['school_name','grade', 'reading_score']][school_data_complete['grade']=='11th'].groupby('school_name').mean()
senior_rdn = school_data_complete[['school_name','grade', 'reading_score']][school_data_complete['grade']=='12th'].groupby('school_name').mean()

In [43]:
freshmen_rdn = freshmen_rdn.rename(columns={'reading_score': '9th Grade Avg Reading Score'})
sphomore_rdn = sphomore_rdn.rename(columns={'reading_score': '10th Grade Avg Reading Score'})
junior_rdn = junior_rdn.rename(columns={'reading_score': '11th Grade Avg Reading Score'})
senior_rdn = senior_rdn.rename(columns={'reading_score': '12th Grade Avg Reading Score'})

Avg_rdn_by_school = pd.merge(freshmen_rdn, sphomore_rdn, on='school_name')
Avg_rdn_by_school = pd.merge(Avg_rdn_by_school, junior_rdn, on='school_name')
Avg_rdn_by_school = pd.merge(Avg_rdn_by_school, senior_rdn, on='school_name')

Avg_rdn_by_school['9th Grade Avg Reading Score'] = Avg_rdn_by_school['9th Grade Avg Reading Score'].map('{:.2f}'.format)
Avg_rdn_by_school['10th Grade Avg Reading Score'] = Avg_rdn_by_school['10th Grade Avg Reading Score'].map('{:.2f}'.format)
Avg_rdn_by_school['11th Grade Avg Reading Score'] = Avg_rdn_by_school['11th Grade Avg Reading Score'].map('{:.2f}'.format)
Avg_rdn_by_school['12th Grade Avg Reading Score'] = Avg_rdn_by_school['12th Grade Avg Reading Score'].map('{:.2f}'.format)

In [44]:
Avg_rdn_by_school

,9th Grade Avg Reading Score,10th Grade Avg Reading Score,11th Grade Avg Reading Score,12th Grade Avg Reading Score
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [45]:
# Sample bins. Feel free to create your own bins.
spending_bins = [school_data_complete['budget'].min()-1, 1000000, 1500000, 2000000, 3000000, school_data_complete['budget'].max()]
group_names = ["200k-1000K", "$1000K-1500K", "$1500K-2000K", "$2500K-3000K","$3000K+"]

spending_bins

[248086, 1000000, 1500000, 2000000, 3000000, 3124928]

In [46]:
by_budget = school_data_complete.groupby(pd.cut(school_data_complete['budget'], spending_bins, labels = group_names))

In [47]:
# slice data set into ranges
range1 = school_data_complete[(school_data_complete['budget']>200000) & (school_data_complete['budget']<=1000000)]
range2 = school_data_complete[(school_data_complete['budget']>1000000) & (school_data_complete['budget']<=1500000)]
range3 = school_data_complete[(school_data_complete['budget']>1500000) & (school_data_complete['budget']<=2000000)]
range4 = school_data_complete[(school_data_complete['budget']>2000000) & (school_data_complete['budget']<=3000000)]
range5 = school_data_complete[(school_data_complete['budget']>3000000) & (school_data_complete['budget']<=school_data_complete['budget'].max())]


In [48]:
# get count of pass from each range
range1_math_pass = range1[range1['math_score']>=70]['Student ID'].count()
range2_math_pass = range2[range2['math_score']>=70]['Student ID'].count()
range3_math_pass = range3[range3['math_score']>=70]['Student ID'].count()
range4_math_pass = range4[range4['math_score']>=70]['Student ID'].count()
range5_math_pass = range5[range5['math_score']>=70]['Student ID'].count()

range1_rdn_pass = range1[range1['reading_score']>=70]['Student ID'].count()
range2_rdn_pass = range2[range2['reading_score']>=70]['Student ID'].count()
range3_rdn_pass = range3[range3['reading_score']>=70]['Student ID'].count()
range4_rdn_pass = range4[range4['reading_score']>=70]['Student ID'].count()
range5_rdn_pass = range5[range5['reading_score']>=70]['Student ID'].count()

In [49]:
# create dataframe with budget ranges and pass counts for math and reading
pass_count = pd.DataFrame({'math pass': [range1_math_pass, range2_math_pass, range3_math_pass, range4_math_pass, range5_math_pass],
              'reading pass': [range1_rdn_pass, range2_rdn_pass, range3_rdn_pass, range4_rdn_pass, range5_rdn_pass],
              'budget': ["200k-1000K", "$1000K-1500K", "$1500K-2000K", "$2500K-3000K", "$3000K+"]
             }).set_index('budget')

score_count = by_budget[['math_score','reading_score']].count().rename(columns={'math_score': 'math count',
                                                                  'reading_score': 'reading count'})

avg_score =by_budget[['math_score','reading_score']].mean()

In [50]:
summary_by_budget = pd.merge(avg_score, pass_count, on='budget')
summary_by_budget = pd.merge(summary_by_budget, score_count, on='budget')

summary_by_budget['% Passing Math'] = 100*(summary_by_budget['math pass']/summary_by_budget['math count'])
summary_by_budget['% Passing Reading'] = 100*(summary_by_budget['reading pass']/summary_by_budget['reading count'])
summary_by_budget['% Overall Passing Rate']= (summary_by_budget['% Passing Math'] + summary_by_budget['% Passing Reading'])/2

In [51]:
#summary_by_budget

In [52]:
del summary_by_budget['reading pass']
del summary_by_budget['math pass']
del summary_by_budget['reading count']
del summary_by_budget['math count']

#summary_by_budget


In [53]:
summary_by_budget.rename(columns={'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
budget,,,,,
200k-1000K,83.583479,83.893245,93.664683,96.604830,95.134757
$1000K-1500K,83.351933,83.905751,93.713184,96.658456,95.185820
$1500K-2000K,76.808251,81.035328,66.624056,80.476467,73.550261
$2500K-3000K,76.842711,80.744686,66.366592,80.220055,73.293323
$3000K+,77.134219,80.979474,66.497356,81.352630,73.924993


## Scores by School Size

* Perform the same operations as above, based on school size.

In [54]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, school_data_complete['size'].max()]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (>2000)"]

In [55]:
by_size = school_data_complete.groupby(pd.cut(school_data_complete['size'], size_bins, labels = group_names))

In [56]:
# slice data set into ranges
range1 = school_data_complete[(school_data_complete['size']>0) & (school_data_complete['size']<=1000)]
range2 = school_data_complete[(school_data_complete['size']>1000) & (school_data_complete['size']<=2000)]
range3 = school_data_complete[(school_data_complete['size']>2000) & (school_data_complete['size']<=school_data_complete['size'].max())]


In [57]:
# get count of pass from each range
range1_math_pass = range1[range1['math_score']>=70]['Student ID'].count()
range2_math_pass = range2[range2['math_score']>=70]['Student ID'].count()
range3_math_pass = range3[range3['math_score']>=70]['Student ID'].count()

range1_rdn_pass = range1[range1['reading_score']>=70]['Student ID'].count()
range2_rdn_pass = range2[range2['reading_score']>=70]['Student ID'].count()
range3_rdn_pass = range3[range3['reading_score']>=70]['Student ID'].count()

In [58]:
pass_count = pd.DataFrame({'math pass': [range1_math_pass, range2_math_pass, range3_math_pass],
                           'reading pass': [range1_rdn_pass, range2_rdn_pass, range3_rdn_pass],
                           'size': group_names
                          }).set_index('size')

score_count = by_size[['math_score','reading_score']].count().rename(columns={'math_score': 'math count',
                                                                  'reading_score': 'reading count'})

avg_score = by_size[['math_score','reading_score']].mean()

In [59]:
avg_score

,math_score,reading_score
size,,
Small (<1000),83.828654,83.974082
Medium (1000-2000),83.372682,83.867989
Large (>2000),77.477597,81.198674


In [60]:
summary_by_size = pd.merge(avg_score, pass_count, on='size')
summary_by_size = pd.merge(summary_by_size, score_count, on='size')

summary_by_size['% Passing Math'] = 100*(summary_by_size['math pass']/summary_by_size['math count'])
summary_by_size['% Passing Reading'] = 100*(summary_by_size['reading pass']/summary_by_size['reading count'])
summary_by_size['% Overall Passing Rate']= (summary_by_size['% Passing Math'] + summary_by_size['% Passing Reading'])/2

#summary_by_size

In [61]:
del summary_by_size['reading pass']
del summary_by_size['math pass']
del summary_by_size['reading count']
del summary_by_size['math count']

In [62]:
summary_by_size.rename(columns={'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (>2000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [63]:
#school_data_complete.head()

In [64]:
avg_score_by_type = school_data_complete.groupby('type').mean()[['math_score','reading_score' ]].rename(columns={'math_score': 'Average Math Score',
                                                                                                                 'reading_score': 'Average Reading Score'})

In [65]:
rdn_pass_by_type = pd.DataFrame(school_data_complete[school_data_complete['reading_score']>=70].groupby('type').count()['Student ID'])
rdn_pass_by_type = rdn_pass_by_type.rename(columns={'Student ID': 'reading pass count'})


In [66]:
math_pass_by_type = pd.DataFrame(school_data_complete[school_data_complete['math_score']>=70].groupby('type').count()['Student ID'])
math_pass_by_type = math_pass_by_type.rename(columns={'Student ID': 'math pass count'})


In [67]:
total_students = pd.DataFrame(school_data_complete.groupby('type').count()['Student ID'])
total_students = total_students.rename(columns={'Student ID': 'count'})

In [68]:
summary_by_type = pd.merge(avg_score_by_type, math_pass_by_type, on='type')
summary_by_type = pd.merge(summary_by_type, rdn_pass_by_type, on='type')
summary_by_type = pd.merge(summary_by_type, total_students, on='type')
summary_by_type

,Average Math Score,Average Reading Score,math pass count,reading pass count,count
type,,,,,
Charter,83.406183,83.902821,11426,11785,12194
District,76.987026,80.962485,17944,21825,26976


summary_by_type[]

In [69]:
summary_by_type['% Passing Math'] = 100*(summary_by_type['math pass count']/summary_by_type['count'])
summary_by_type['% Passing Reading'] = 100*(summary_by_type['reading pass count']/summary_by_type['count'])
summary_by_type['% Overall Passing Rate'] = (summary_by_type['% Passing Math'] + summary_by_type['% Passing Reading'])/2

In [70]:
del summary_by_type['math pass count']
del summary_by_type['reading pass count']
del summary_by_type['count']
summary_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
